In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import math
import os 

In [ ]:
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import Kan_NN
import importlib
importlib.reload(Kan_NN)

In [ ]:
##### Problem
in_dim = 8
degree = 2
h = [16]

In [ ]:
import random
random.seed(42)
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load dataset
california_housing = fetch_california_housing(as_frame=True)
X = california_housing.data.values  # Convert DataFrame to NumPy array
y = california_housing.target.values.reshape(-1, 1)  # Reshape target to match PyTorch format

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features for better training stability
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders with batch size
batch_size = 1024  # Set batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
batch_test = 1
test_loader = DataLoader(test_dataset, batch_size=batch_test, shuffle=False)


# Check a sample batch
for batch in train_loader:
    X_batch, y_batch = batch
    print(f"Batch X shape: {X_batch.shape}, Batch y shape: {y_batch.shape}")
    break  # Print only one batch for verification


In [ ]:
@torch.no_grad()
def compute_test_loss(test_loader, model):
    criterion = torch.nn.MSELoss()
    running_loss = 0.
    for batch, target in test_loader:
        outputs = model(batch)
        loss = criterion(target, outputs)
        running_loss += loss.item()
    return running_loss / len(test_loader)

In [ ]:
import time
models = []
parameters = []
train_losses = []
test_losses = []
max_depth = 2
width = 1
for depth in range(1,max_depth):
    mods = []
    pars = []
    for i in range(width):
        shape = [8] + [2**i for _ in range(j)] + [1]
        model = Kan_NN.Neural_Kan(shape = shape, h = [32])
        print(shape, model.params)
        pars.append(model.params)
        epochs = 1
        train_loss = []
        test_loss = []
        model.train()
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0)
        criterion = torch.nn.MSELoss()
        for epoch in range(epochs):
            running_loss = 0.0 
            for batch, target in train_loader:
                start_time = time.time()
                optimizer.zero_grad()
                outputs = model(batch)
                loss = criterion(target, outputs)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            avg_loss = running_loss / len(train_loader)
            train_loss.append(avg_loss)
            test_l = compute_test_loss(test_loader, model)
            test_loss.append(test_l)
            print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.6f}, test: {test_l:.6f}, lr: {optimizer.param_groups[0]['lr']:6f}")
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        mods.append(model)
        plt.plot(train_loss[-50:])
        plt.title(f'train_loss')
        plt.legend()
        plt.show()
        print("Training Complete!")
    models.append(mods)
    parameters.append(pars)


In [ ]:
import dill 
with open("CC_house_kan.dill", "wb") as f:
    dill.dump(models, f)